In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import string
import json
import re
import os
import sys
import pip
import keras
import time
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from keras import initializers as initializers, regularizers, constraints
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Sequential, Model, load_model
from keras.preprocessing.sequence import pad_sequences # To make vectors the same size. 
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, GlobalMaxPool1D, MaxPool1D,concatenate
from keras.optimizers import Adam
from keras.callbacks import CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.engine.topology import Layer
from keras.utils import plot_model, np_utils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [10]:
emotions_new = np.load("/home/alissa77/WWW2021 copy/code/preprocess/data/Weibo-16/emotions_new/val_(738, 55).npy")
emotions = np.load("/home/alissa77/WWW2021 copy/code/preprocess/data/Weibo-16/emotions/val_(738, 55).npy")
cvaw_weibo = np.load("/home/alissa77/WWW2021 copy/code/preprocess/data_cvaw_weibo/Weibo-16/emotions/val_(738, 55).npy")
#emotions_new2 = np.load("/home/alissa77/WWW2021 copy/code/preprocess/data/Weibo-20/emotions_new/Weibo-20_test_(1272, 275).npy")

# prepare sentiment feature

In [19]:
# mine method + cvaw resource
datasets_ch = ['Weibo-16', 'Weibo-16-original',
               'Weibo-20', 'Weibo-20-temporal']
dataset_dir = '/home/alissa77/WWW2021/code/preprocess/data'
dataset_dir_new = '/home/alissa77/fake-news-detect/preprocess_data'
save_dir = '../data'
tmp = []
dic2 = {}

if not os.path.exists(save_dir):
        os.mkdir(save_dir)

for dataset in datasets_ch : 
    print('\n\n{} [{}]\tProcessing the dataset: {} {}\n'.format(
    '-'*20, time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()), dataset, '-'*20))
    
    dic = {}
    output_dir = os.path.join(save_dir, dataset)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    emotion_dir = os.path.join(output_dir, 'emotions_new')
    if not os.path.exists(emotion_dir):
        os.mkdir(emotion_dir)

    data_dir_new = os.path.join(dataset_dir_new, dataset)  
    for t in ['train', 'val', 'test']:
        name1 = data_dir_new.split('/')[5]+"_"+t
        dic2[name1] = pd.read_pickle(os.path.join(data_dir_new, '{}.pkl'.format(t)))['sentiment_score'] 

    for t in ['emotions']:
        data_dir = os.path.join(dataset_dir, dataset, t)
        for f in os.listdir(data_dir):
            f = os.path.join(data_dir, f)
            name = ''
            if 'train_' in f:
                name = f.split('/')[7]+"_train"
                dic[name] = np.load(f)
            elif 'val_' in f:
                name = f.split('/')[7]+"_val"
                dic[name] = np.load(f)
            elif 'test_' in f:
                name = f.split('/')[7]+"_test"
                dic[name] = np.load(f)
        
    for arr in dic.keys():
        dic[arr][:,37:38] = dic2[arr][:,np.newaxis] # shape(3) -> (3,1) 
        print(arr)
        np.save(os.path.join(emotion_dir, '{}_{}.npy'.format(arr, dic[arr].shape)), dic[arr])



-------------------- [2021-08-11 16:23:39]	Processing the dataset: Weibo-16 --------------------

Weibo-16_val
Weibo-16_test
Weibo-16_train


-------------------- [2021-08-11 16:23:46]	Processing the dataset: Weibo-16-original --------------------

Weibo-16-original_val
Weibo-16-original_test
Weibo-16-original_train


-------------------- [2021-08-11 16:23:54]	Processing the dataset: Weibo-20 --------------------



KeyboardInterrupt: 

# change the sentiment feature

In [42]:
# 他的方法用在我的cvaw

import joblib
import pandas as pd
import numpy as np
import jieba
import csv




# negation value and degree value
def get_not_and_how_value(cut_words, i, windows):
    not_cnt = 0
    how_v = 1
    left = 0 if (i - windows) < 0 else (i - windows)
    window_text = ' '.join(cut_words[left:i])
    for w in negation_words:
        if w in window_text:
            not_cnt += 1
    for w in how_words_dict.keys():
        if w in window_text:
            how_v *= how_words_dict[w]

    return (-1) ** not_cnt, how_v


def boson_value(cut_words, windows=2):
    def b_edge(value):
        if value<0:
            return -4
        else:
            return 4
    valence = {}
    arousal = {}
    with open('/home/alissa77/WWW2021 copy/code/emotion/cvaw4.csv', newline='') as csvfile:
        rows = csv.DictReader(csvfile)
        for row in rows:
            valence[row['Word']] = (row['Valence_Mean'])
            arousal[row['Word']] = (row['Arousal_Mean'])
            
    valence = {x: round(float(valence[x])-5, 2) for x in valence}
    arousal = {x: round(float(arousal[x])-5, 2) for x in arousal}
    print('[CVAW]\t There are {} words'.format(len(arousal)))

    for ID in cut_words:
        score_v = 0
        score_a = 0
        score_degree = 0
        score_negative = 0
        len_cvaw = 1 

        for i in range(2, len(cut_words[ID])): # lyric[i] = a word in CVAW dictionary
            lyric = cut_words[ID]
            temp_v = 0  # vep
            temp_a = 0  # aep

            if  cut_words[ID] in list(degree):
                score_degree += degree[ cut_words[ID]]
            if  cut_words[ID] in list(negative):
                score_negative += negative[ cut_words[ID]]


    value = 0
    for i, word in enumerate(cut_words):
        if word in arousal:
            not_v, how_v = get_not_and_how_value(cut_words, i, windows)
            
            value += not_v * how_v * arousal[word] #compute right here!!!!!!

    return value


# def extract_publisher_emotion(content, content_words, emotions_dict):
#     text, cut_words = content, content_words
#     arr = np.zeros(2)
#     arr[:1] = boson_value(cut_words)

cut_wrds = ['我','好','想','睡']
boson_value(cut_wrds)

[CVAW]	 There are 5512 words


TypeError: list indices must be integers or slices, not str

In [101]:
datasets_ch = ['Weibo-16', 'Weibo-16-original',
               'Weibo-20', 'Weibo-20-temporal']
dataset_dir = '/home/alissa77/WWW2021/code/preprocess/data'
dataset_dir_new = '/home/alissa77/fake-news-detect/preprocess_data'
tmp = []
dic = {}
dic2 = {}
arrs = ['train', 'val', 'test']

## data_a
for dataset in datasets_ch : 
    train_data, val_data, test_data = [], [], []
    data_dir_new = os.path.join(dataset_dir_new, dataset)  
    for t in ['train', 'val', 'test']:
        name1 = data_dir_new.split('/')[5]+"_"+t
        dic2[name1] = pd.read_pickle(os.path.join(data_dir_new, '{}.pkl'.format(t)))['sentiment_score'] 
        print(dic2[name1])

0       0.6
1       0.6
2       1.2
3       0.0
4       0.0
       ... 
2206    2.0
2207   -0.6
2208    1.0
2209    0.0
2210    0.0
Name: sentiment_score, Length: 2211, dtype: float64
0      3.40
1      0.00
2      1.60
3      5.60
4     -0.80
       ... 
733   -0.40
734   -0.28
735    2.00
736    1.44
737    0.00
Name: sentiment_score, Length: 738, dtype: float64
0      0.0
1      0.8
2      0.0
3      4.8
4      3.8
      ... 
752    2.8
753    0.0
754    0.2
755   -0.8
756    0.8
Name: sentiment_score, Length: 757, dtype: float64
0       0.60
1       0.60
2       4.96
3       1.20
4       0.00
        ... 
2791    2.00
2792   -0.60
2793    1.00
2794    0.00
2795    0.00
Name: sentiment_score, Length: 2796, dtype: float64
0      0.00
1      0.00
2      0.00
3      0.00
4      3.40
       ... 
928   -0.40
929   -0.28
930    2.00
931    1.44
932    0.00
Name: sentiment_score, Length: 933, dtype: float64
0      0.0
1     -8.0
2      0.0
3      5.6
4      0.0
      ... 
929    2.8
930   

In [24]:
import pickle
dataset_dir_new = '/home/alissa77/fake-news-detect/preprocess_data'

for dataset in datasets_ch : 
    data_dir_new = os.path.join(dataset_dir_new, dataset)  
    split_datasets = [pd.read_pickle(os.path.join(data_dir_new, '{}.pkl'.format(t))) for t in ['train', 'val', 'test']] 
    split_datasets = dict(zip(['train', 'val', 'test'], split_datasets))
    for t, pieces in split_datasets.items(): 
        print(pieces['sentiment_score'])

0       0.6
1       0.6
2       1.2
3       0.0
4       0.0
       ... 
2206    2.0
2207   -0.6
2208    1.0
2209    0.0
2210    0.0
Name: sentiment_score, Length: 2211, dtype: float64
0      3.40
1      0.00
2      1.60
3      5.60
4     -0.80
       ... 
733   -0.40
734   -0.28
735    2.00
736    1.44
737    0.00
Name: sentiment_score, Length: 738, dtype: float64
0      0.0
1      0.8
2      0.0
3      4.8
4      3.8
      ... 
752    2.8
753    0.0
754    0.2
755   -0.8
756    0.8
Name: sentiment_score, Length: 757, dtype: float64
0       0.60
1       0.60
2       4.96
3       1.20
4       0.00
        ... 
2791    2.00
2792   -0.60
2793    1.00
2794    0.00
2795    0.00
Name: sentiment_score, Length: 2796, dtype: float64
0      0.00
1      0.00
2      0.00
3      0.00
4      3.40
       ... 
928   -0.40
929   -0.28
930    2.00
931    1.44
932    0.00
Name: sentiment_score, Length: 933, dtype: float64
0      0.0
1     -8.0
2      0.0
3      5.6
4      0.0
      ... 
929    2.8
930   

In [3]:
MAX_SENTENCE_NUM = 150 # maximum number of sentences in one document
MAX_WORD_NUM = 150     # maximum number of words in each sentence
EMBED_SIZE = 768      # vector size of word embedding

In [65]:
from keras.layers import Input
from keras.layers import Bidirectional
from keras.layers import GRU, TimeDistributed
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dense
from keras.layers import Embedding, Concatenate
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.initializers import Constant
from keras import backend as K
from keras.layers.core import Lambda


class EmotionEnhancedBiGRU:
    def __init__(self, max_sequence_length, embedding_matrix, emotion_dim=0, category_num=2, hidden_units=32, l2_param=0.01, lr_param=0.001):
        self.max_sequence_length = max_sequence_length
        self.embedding_matrix = embedding_matrix
        self.emotion_dim = emotion_dim
        self.hidden_units = hidden_units
        self.category_num = category_num
        self.l2_param = l2_param

        self.model = self.build()
        self.model.compile(loss='binary_crossentropy', optimizer=Adam(
            lr=lr_param, beta_1=0.8), metrics=['accuracy'])

    def build(self):
        semantic_input = Input(
            shape=(self.max_sequence_length,), name='word-embedding-input')

        semantic_emb = Embedding(self.embedding_matrix.shape[0],
                                 self.embedding_matrix.shape[1],
                                 embeddings_initializer=Constant(
                                     self.embedding_matrix),
                                 input_length=self.max_sequence_length,
                                 trainable=False)(semantic_input)

        gru = Bidirectional(
            GRU(self.hidden_units, return_sequences=True))(semantic_emb)
        avg_pool = GlobalAveragePooling1D()(gru)

        if self.emotion_dim != 0:
            emotion_input = Input(
                shape=(self.emotion_dim,), name='emotion-input')
            emotion_enhanced = Concatenate()([avg_pool, emotion_input])

            dense = Dense(32, activation='relu', kernel_regularizer=l2(
                self.l2_param))(emotion_enhanced)
            output = Dense(self.category_num, activation='softmax',
                           kernel_regularizer=l2(self.l2_param))(dense)
            model = Model(
                inputs=[semantic_input, emotion_input], outputs=output)
        else:
            dense = Dense(32, activation='relu',
                          kernel_regularizer=l2(self.l2_param))(avg_pool)
            output = Dense(self.category_num, activation='softmax',
                           kernel_regularizer=l2(self.l2_param))(dense)
            model = Model(inputs=[semantic_input], outputs=output)

        return model

In [66]:
def cleanString(review,stopWords):
    lemmatizer = WordNetLemmatizer()
    returnString = ""
    sentence_token = tokenize.sent_tokenize(review)
    idx_list = []
    for j in range(len(sentence_token)):
        single_sentence = tokenize.word_tokenize(sentence_token[j])
        sentences_filtered = [(idx,lemmatizer.lemmatize(w.lower())) for idx,w in enumerate(single_sentence) 
                              if w.lower() not in stopWords and w.isalnum()]
        idx_list.append([x[0] for x in sentences_filtered])
        word_list = [x[1] for x in sentences_filtered]
        returnString = returnString + ' '.join(word_list) + ' . '
    
    return returnString, idx_list


def split_df(dataframe, column_name, training_split = 0.6, validation_split = 0.2, test_split = 0.2):
    """
    Splits a pandas dataframe into trainingset, validationset and testset in specified ratio.
    All sets are balanced, which means they have the same ratio for each categorie as the full set.
    Input:   dataframe        - Pandas Dataframe, should include a column for data and one for categories
             column_name      - Name of dataframe column which contains the categorical output values
             training_split   - from ]0,1[, default = 0.6
             validation_split - from ]0,1[, default = 0.2        
             test_split       - from ]0,1[, default = 0.2
                                Sum of all splits need to be 1
    Output:  train            - Pandas DataFrame of trainset
             validation       - Pandas DataFrame of validationset
             test             - Pandas DataFrame of testset
    """
    if training_split + validation_split + test_split != 1.0:
        raise ValueError('Split paramter sum should be 1.0')
        
    total = len(dataframe.index)
 
    train = dataframe.reset_index().groupby(column_name).apply(lambda x: x.sample(frac=training_split))\
    .reset_index(drop=True).set_index('index')
    train = train.sample(frac=1)
    temp_df = dataframe.drop(train.index)
    validation = temp_df.reset_index().groupby(column_name)\
    .apply(lambda x: x.sample(frac=validation_split/(test_split+validation_split)))\
           .reset_index(drop=True).set_index('index')
    validation = validation.sample(frac=1)
    test = temp_df.drop(validation.index)
    test = test.sample(frac=1)
    
    print('Total: ', len(dataframe))
    print('Training: ', len(train), ', Percentage: ', len(train)/len(dataframe))
    print('Validation: ', len(validation), ', Percentage: ', len(validation)/len(dataframe))
    print('Test:', len(test), ', Percentage: ', len(test)/len(dataframe))

    return train, validation, test

def wordToSeq(text,word_index,max_sentences,max_words,max_features):
    """
    Converts a string to a numpy matrix where each word is tokenized.
    Arrays are zero-padded to max_sentences and max_words length.
    
    Input:    text           - string of sentences
              word_index     - trained word_index
              max_sentences  - maximum number of sentences allowed per document for HAN
              max_words      - maximum number of words in each sentence for HAN
              max_features   - maximum number of unique words to be tokenized
    Output:   data           - Numpy Matrix of size [max_sentences x max_words]
    """
    sentences = tokenize.sent_tokenize(text)
    data = np.zeros((max_sentences, max_words), dtype='int32')
    for j, sent in enumerate(sentences):
        if j< max_sentences:
            wordTokens = tokenize.word_tokenize(sent.rstrip('.'))
            wordTokens = [w for w in wordTokens]
            k=0
            for _, word in enumerate(wordTokens):
                try:
                    if k<max_words and word_index[word]<max_features:
                        data[j,k] = word_index[word]
                        k=k+1
                except:
                    pass
    return data


class AttentionLayer(Layer):
    def __init__(self,attention_dim=768,return_coefficients=False,**kwargs):
    #def __init__(self,attention_dim=100,return_coefficients=False,**kwargs):
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

# sentiment

In [67]:
#test_lyrics = df.WS.to_dict()
#train_lyrics = np.load("train_WS.npy",allow_pickle=True)

# 程度副詞 #md
degree = {'最':0.9, '最為':0.9, '極':0.9, '極為':0.9, '極其':0.9, '極度':0.9, '極端':0.9, '至為':0.9,\
           '頂':0.9, '過':0.9, '過於':0.9, '過分':0.9, '分外':0.9, '萬分':0.9,"全":0.9,\
           '更':0.7, '更加':0.7, '更為':0.7, '更其':0.7, '越':0.7, '越發':0.7, '備加':0.7, '愈加':0.7, '愈':0.7,\
           '愈發':0.7, '愈為':0.7, '愈益':0.7, '越加':0.7, '格':0.7, '益發':0.7, '還':0.7, '很':0.7, '太':0.7,'都':0.7,\
           '挺':0.7, '怪':0.7, '老是':0.7, '非常':0.7, '特別':0.7, '相當':0.7, '十分':0.7, '好':0.7, '好不':0.9,\
           '甚':0.7, '甚為':0.7, '頗':0.7, '頗為':0.7, '滿':0.7, '蠻':0.7, '夠':0.7, '多':0.7,\
           '多麼':0.7, '特':0.7, '大':0.7, '大為':0.7, '何等':0.7, '何其':0.7, '尤其':0.7, '無比':0.7, '尤為':-0.7,\
           '較':0.5, '比較':0.5, '較比':0.5, '較為':0.5, '還':0.5, '不大':-0.5, '不太':0.5, '不很':0.5, '不甚':0.5,\
           '早已':0.7,'嗎':0.5,'大多':-0.5,'超級':0.7,\
           '稍':-0.5, '稍稍':-0.5, '稍為':-0.5, '稍微':-0.5, '稍許':-0.5, '親自':-0.5, '略':-0.5, '略為':-0.5,\
           '些微':-0.5, '多少':-0.5, '有點':-0.7, '有點兒':-0.5, '有些':-0.5, '多為':-0.7,'！':0.7}

# 否定副詞 #mn
negative = {'白':-0.8, '白白':-0.8, '甭':-0.8, '別':-0.8, '不':-0.8, '不必':-0.8, '不曾':-0.8, '不太':-0.8,'不要':-0.8,\
          '不用':-0.8, '非':-0.8, '幹':-0.8, '何必':-0.8, '何曾':-0.8, '何嘗':-0.8, '何須':-0.8,\
          '空':-0.8, '沒':-0.8, '沒有':-0.8, '莫':-0.8, '徒':-0.8, '徒然':-0.8, '忹':-0.8,\
          '未':-0.8, '未曾':-0.8, '未嘗':-0.8, '無須':-0.8, '無須乎':-0.8, '無需':-0.8, '毋須':-0.8,\
          '毋庸':-0.8, '無庸':-0.8, '勿':-0.8, '瞎':-0.8, '休':-0.8, '虛':-0.8}

In [68]:
import csv 

def sesntiment_score(lyrics,dataframe):
    def b_edge(value):
        if value<0:
            return -4
        else:
            return 4
    lst=[]
    valence = {}
    arousal = {}
    with open('cvaw4.csv', newline='') as csvfile:
        rows = csv.DictReader(csvfile)
        for row in rows:
            valence[row['Word']] = (row['Valence_Mean'])
            arousal[row['Word']] = (row['Arousal_Mean'])

    valence = {x: round(float(valence[x])-5, 2) for x in valence}
    arousal = {x: round(float(arousal[x])-5, 2) for x in arousal}

    for ID in lyrics:
        score_v = 0
        score_a = 0
        score_degree = 0
        score_negative = 0
        len_cvaw = 1 

        for i in range(2, len(lyrics[ID])): # lyric[i] = a word in CVAW dictionary
            lyric = lyrics[ID]
            temp_v = 0  # vep
            temp_a = 0  # aep

            if  lyrics[ID] in list(degree):
                score_degree += degree[ lyrics[ID]]
            if  lyrics[ID] in list(negative):
                score_negative += negative[ lyrics[ID]]

            if lyric[i] in arousal:
                len_cvaw += 1
                this_v = valence[lyric[i]]  # Vew
                this_a = arousal[lyric[i]]  # Aew


                if lyric[i-2] in negative:
                    if lyric[i-1] in negative:
                        print ("N + N + EW:", lyric[i-2:i+1:])
                        param = negative[ lyric[i-2] ] * negative[ lyric[i-1] ]
                        temp_v = param * this_v
                        temp_a = param * this_a

                    elif lyric[i-1] in degree:
                        print ("N + D + EW:", lyric[i-2:i+1:])
                        param = degree[ lyric[i-1] ] - (1 + negative[ lyric[i-2] ])
                        temp_v = this_v + (b_edge(this_v) - this_v) * param
                        temp_a = this_a + (b_edge(this_a) - this_a) * param

                elif lyric[i-2] in degree:
                    if lyric[i-1] in negative:
                        print ("D + N + EW:", lyric[i-2:i+1:])
                        mn = negative[ lyric[i-1] ]
                        md = degree[ lyric[i-2] ]
                        param_v = mn * this_v
                        param_a = mn * this_a
                        temp_v = param_v + (b_edge(this_v) - param_v) * md
                        temp_a = param_a + (b_edge(this_a) - param_a) * md

                    elif lyric[i-1] in degree:
                        print ("D + D + EW:", lyric[i-2:i+1:])
                        md_1 = degree[ lyric[i-1] ]
                        md_2 = degree[ lyric[i-2] ]
                        param_v = (b_edge(this_v) - this_v) * md_1
                        param_a = (b_edge(this_a) - this_a) * md_1
                        temp_v = this_v + param_v + (1 - (this_v + param_v)) * md_2
                        temp_a = this_a + param_a + (1 - (this_a + param_a)) * md_2

                elif lyric[i-1] in negative:
                    print ("N + EW:", lyric[i-1:i+1:])
                    temp_v = negative[ lyric[i-1] ] * valence[lyric[i]]
                    temp_a = negative[ lyric[i-1] ] * arousal[lyric[i]]

                elif lyric[i-1] in degree:
                    print ("D + EW:", lyric[i-1:i+1:])
                    temp_v = this_v + (b_edge(this_v) - this_v) * degree[ lyric[i-1] ]
                    temp_a = this_a + (b_edge(this_a) - this_a) * degree[ lyric[i-1] ]
                
                else:
                    print ("EW:", lyric[i])   
                    temp_v = valence[lyric[i]]
                    temp_a = arousal[lyric[i]]
                

            score_v += temp_v
            score_a += temp_a     

        valence_output = score_v/len_cvaw
        
        # 排除情況（第四象限
        if (score_a < 0 and valence_output == 0):
            score_a = 0         
        
        lst.append(score_a)
        new_emotion_word = pd.Series(lst)   # add new emotion list
        dataframe['sentiment_score'] = new_emotion_word
        
        #print ("Mean_valence:", valence_output)
        print(lyrics[ID])
        print ("Mean_arousal:", score_a)
        print("--------------------------")

In [79]:
#padding

train_ss = np.asarray(list(data["sentiment_score"]))
test_ss = np.asarray(list(df["sentiment_score"]))
train_ss = np.expand_dims(train_ss, axis=1)
test_ss = np.expand_dims(test_ss, axis=1)
#train_ss = np.expand_dims(y, axis=2)
#test_ss = np.expand_dims(y1, axis=2)
#train_ss = pad_sequences(x, maxlen = 150, padding = 'post',dtype = "float32")
#test_ss = pad_sequences(x1, maxlen = 150, padding = 'post',dtype = "float32")
print(test_ss.shape)
print(train_ss.shape)

(1033, 1)
(4222, 1)


In [80]:
test_ss

array([[0.6],
       [0. ],
       [0.8],
       ...,
       [1.6],
       [0.8],
       [0. ]])

In [ ]:
import joblib
import pandas as pd
import numpy as np
import jieba
import os
import json
import sys

datasets_ch = ['Weibo-16', 'Weibo-16-original',
               'Weibo-20', 'Weibo-20-temporal']

# ============================== Category ==============================
baidu_emotions = ['angry', 'disgusting', 'fearful',
                  'happy', 'sad', 'neutral', 'pessimistic', 'optimistic']
baidu_emotions.sort()

baidu_emotions_2_index = dict(zip(baidu_emotions, [i for i in range(len(baidu_emotions))]))


def baidu_arr(emotions_dict):
    arr = np.zeros(len(baidu_emotions))

    if emotions_dict is None:
        return arr

    for k, v in emotions_dict.items():
        # like -> happy
        if k == 'like':
            arr[baidu_emotions_2_index['happy']] += v
        else:
            arr[baidu_emotions_2_index[k]] += v

    return arr



# load negation words
negation_words = []
with open('/home/alissa77/WWW2021/resources/Chinese/others/negative/negationWords.txt', 'r') as src:
    lines = src.readlines()
    for line in lines:
        negation_words.append(line.strip())

print('\nThe num of negation words: ', len(negation_words))

# load degree words
how_words_dict = dict()
with open('/home/alissa77/WWW2021/resources/Chinese/HowNet/intensifierWords.txt', 'r') as src:
    lines = src.readlines()
    for line in lines:
        how_word = line.strip().split()
        how_words_dict[' '.join(how_word[:-1])] = float(how_word[-1])

print('The num of degree words: ', len(how_words_dict),
      '. eg: ', list(how_words_dict.items())[0])


# negation value and degree value
def get_not_and_how_value(cut_words, i, windows):
    not_cnt = 0
    how_v = 1

    left = 0 if (i - windows) < 0 else (i - windows)
    window_text = ' '.join(cut_words[left:i])

    for w in negation_words:
        if w in window_text:
            not_cnt += 1
    for w in how_words_dict.keys():
        if w in window_text:
            how_v *= how_words_dict[w]

    # for w in cut_words[left:i]:
    #     if w in negation_words:
    #         not_cnt += 1
    #     if w in how_words_dict:
    #         how_v *= how_words_dict[w]

    return (-1) ** not_cnt, how_v


_, words2array = joblib.load(
    '/home/alissa77/WWW2021/resources/Chinese/大连理工大学情感词汇本体库/preprocess/words2array_27351.pkl')
print('[Dalianligong]\tThere are {} words, the dimension is {}'.format(
    len(words2array), words2array['快乐'].shape))


def dalianligong_arr(cut_words, windows=2):
    arr = np.zeros(29)

    for i, word in enumerate(cut_words):
        if word in words2array:
            not_v, how_v = get_not_and_how_value(cut_words, i, windows)
            arr += not_v * how_v * words2array[word]

    return arr




# ============================== Auxilary Features ==============================


# Emoticon
emoticon_df = pd.read_csv(
    '/home/alissa77/WWW2021/resources/Chinese/others/emoticon/emoticon.csv')
emoticons = emoticon_df['emoticon'].tolist()
emoticon_types = list(set(emoticon_df['label'].tolist()))
emoticon_types.sort()
emoticon2label = dict(
    zip(emoticon_df['emoticon'].tolist(), emoticon_df['label'].tolist()))
emoticon2index = dict(
    zip(emoticon_types, [i for i in range(len(emoticon_types))]))

print('[Emoticon]\tThere are {} emoticons, including {} categories'.format(
    len(emoticons), len(emoticon_types)))


def emoticon_arr(text, cut_words):
    arr = np.zeros(len(emoticon_types))

    if len(cut_words) == 0:
        return arr

    for i, emoticon in enumerate(emoticons):
        if emoticon in text:
            arr[emoticon2index[emoticon2label[emoticon]]
                ] += text.count(emoticon)

    return arr / len(cut_words)


# Punctuation
def symbols_count(text):
    excl = (text.count('!') + text.count('！')) / len(text)
    ques = (text.count('?') + text.count('？')) / len(text)
    comma = (text.count(',') + text.count('，')) / len(text)
    dot = (text.count('.') + text.count('。')) / len(text)
    ellip = (text.count('..') + text.count('。。')) / len(text)

    return excl, ques, comma, dot, ellip


# Sentimental Words
def init_words(file):
    with open(file, 'r', encoding='utf-8') as src:
        words = src.readlines()
        words = [l.strip() for l in words]
    # print('File: {}, Words_sz = {}'.format(file.split('/')[-1], len(words)))
    return list(set(words))


pos_words = init_words('/home/alissa77/WWW2021/resources/Chinese/HowNet/正面评价词语（中文）.txt')
pos_words += init_words('/home/alissa77/WWW2021/resources/Chinese/HowNet/正面评价词语（中文）.txt')
neg_words = init_words('/home/alissa77/WWW2021/resources/Chinese/HowNet/负面情感词语（中文）.txt')
neg_words += init_words('/home/alissa77/WWW2021/resources/Chinese/HowNet/负面评价词语（中文）.txt')

pos_words = set(pos_words)
neg_words = set(neg_words)
print('[HowNet]\tThere are {} positive words and {} negative words'.format(
    len(pos_words), len(neg_words)))


def sentiment_words_count(cut_words):
    if len(cut_words) == 0:
        return [0, 0, 0, 0]

    # positive and negative words
    sentiment = []
    for words in [pos_words, neg_words]:
        c = 0
        for word in words:
            if word in cut_words:
                # print(word)
                c += 1
        sentiment.append(c)
    sentiment = [c / len(cut_words) for c in sentiment]

    # degree words
    degree = 0
    for word in how_words_dict:
        if word in cut_words:
            # print(word)
            degree += how_words_dict[word]

    # negation words
    negation = 0
    for word in negation_words:
        negation += cut_words.count(word)
    negation /= len(cut_words)

    sentiment += [degree, negation]

    return sentiment


# Personal Pronoun
first_pronoun = init_words(
    '/home/alissa77/WWW2021/resources/Chinese/others/pronoun/1-personal-pronoun.txt')
second_pronoun = init_words(
    '/home/alissa77/WWW2021/resources/Chinese/others/pronoun/2-personal-pronoun.txt')
third_pronoun = init_words(
    '/home/alissa77/WWW2021/resources/Chinese/others/pronoun/3-personal-pronoun.txt')
pronoun_words = [first_pronoun, second_pronoun, third_pronoun]


def pronoun_count(cut_words):
    if len(cut_words) == 0:
        return [0, 0, 0]

    pronoun = []
    for words in pronoun_words:
        c = 0
        for word in words:
            c += cut_words.count(word)
        pronoun.append(c)

    return [c / len(cut_words) for c in pronoun]


# Auxilary Features
def auxilary_features(text, cut_words):
    arr = np.zeros(17)

    arr[:5] = emoticon_arr(text, cut_words)
    arr[5:10] = symbols_count(text)
    arr[10:14] = sentiment_words_count(cut_words)
    arr[14:17] = pronoun_count(cut_words)

    return arr


def extract_publisher_emotion(claim, content_words):
    text, cut_words = claim, content_words

    arr = np.zeros(55)
    arr[:8] = baidu_arr(emotion_dict)
    arr[8:37] = dalianligong_arr(cut_words)
    arr[37:54] = auxilary_features(text, cut_words)
    arr[54:55] = piece['sentiment_score']
    return arr

def extract_dual_emotion(piece):
    single_emotion = extract_publisher_emotion(piece['claim'], piece['content_words'])    
    return single_emotion

# POS

In [27]:
test_POS = np.load("test_POS.npy",allow_pickle=True)
train_POS = np.load("train_POS.npy",allow_pickle=True)
testpos = pd.DataFrame(test_POS, columns = ['head_pos'])
trainpos = pd.DataFrame(train_POS, columns = ['head_pos'])

In [28]:
import multiprocessing
pos_index = {}
idx = 1

for index, rows in trainpos.iterrows() :
    for pos in rows['head_pos'] :
        if pos not in pos_index :
            pos_index[pos] = idx
            idx += 1
            
for index, rows in testpos.iterrows() :
    for pos in rows['head_pos'] :
        if pos not in pos_index :
            pos_index[pos] = idx
            idx += 1

In [30]:
def head_syn2index(head_syn) :
    pos_list = []
    for pos in head_syn :
        pos_list.append(pos_index[pos])
    
    return pos_list


def _apply_df(args):
    df, func, kwargs = args
    if 'axis' in kwargs:
        axis = kwargs.pop('axis')
        return df.apply(func, **kwargs, axis=axis)
    
    return df.apply(func, **kwargs)


def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    if workers == -1:
        workers = multiprocessing.cpu_count() 
    coln = 1    
    if 'coln' in kwargs:
        coln = kwargs.pop('coln')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    series = pd.concat(list(result))

    if coln == 1:
        return series
    elif coln == 2:
        series_0, series_1 = series.apply(lambda x: x[0]), series.apply(lambda x: x[1])
        return series_0, series_1
    elif coln == 3:
        series_0, series_1, series_2 = series.apply(lambda x: x[0]), series.apply(lambda x: x[1]), series.apply(lambda x: x[2])
        
        return series_0, series_1, series_2  
    
    
def trainset_create(head_pos_index, head_max_len, body_max_article_len) : 
    all_head_pos = []
    head_pos_index = pad_sequences(head_pos_index, maxlen = head_max_len, padding = 'post')
    for pos_list in head_pos_index :
        pos_list = np.array(pos_list)
        pos_list = pos_list.reshape(1,150)

        all_head_pos.append(np.repeat(pos_list, repeats=body_max_article_len, axis=0))
    all_head_pos = np.array(all_head_pos)
     
    return all_head_pos

In [31]:
testpos['head_pos_index'] = apply_by_multiprocessing(testpos['head_pos'], head_syn2index, workers = -1)
trainpos['head_pos_index'] = apply_by_multiprocessing(trainpos['head_pos'], head_syn2index, workers = -1)
testpos

,head_pos,head_pos_index
0,"[Nb, VA, VA, VC, VH, VH, Nc, P, Na, VH, Nc, Nb...","[5, 3, 3, 7, 12, 12, 1, 16, 2, 12, 1, 5, 16, 1..."
1,"[Nc, Nc, D, VC, DE, Na, Na, VC, Na, Neu, Nf, V...","[1, 1, 6, 7, 31, 2, 2, 7, 2, 4, 14, 12, 2]"
2,"[Nd, Nd, Nb, WHITESPACE, VH, VH, Neu, Nf, VC, ...","[9, 9, 5, 52, 12, 12, 4, 14, 7, 4, 14, 2]"
3,"[Nd, Na, D, VH, EXCLAMATIONCATEGORY, WHITESPAC...","[9, 2, 6, 12, 27, 52, 12, 12, 4, 1, 52, 7, 4, ..."
4,"[Nd, Nes, Nf, Nb, Na, Na, WHITESPACE, Nb, VC, ...","[9, 15, 14, 5, 2, 2, 52, 5, 7, 21]"
...,...,...
1028,"[Na, Na, Nh, VK, Na, Na, Nep, Nf, Na, Nep, D, ...","[2, 2, 35, 11, 2, 2, 45, 14, 2, 45, 6, 6, 6, 3..."
1029,"[Neu, Na, VA, VA, VF, VC, Na, EXCLAMATIONCATEG...","[4, 2, 3, 3, 24, 7, 2, 27]"
1030,"[Na, A, VC, D, VC, VH, VD, Di, Na]","[2, 30, 7, 6, 7, 12, 13, 37, 2]"
1031,"[Nb, P, Nc, Na, VC, DE, Na, Ncd, V_2, Neu, Nf,...","[5, 16, 1, 2, 7, 31, 2, 36, 28, 4, 14, 2, 2, 3..."


In [39]:
test_head_pos = trainset_create(list(testpos['head_pos_index']),150, 150)
train_head_pos = trainset_create(list(trainpos['head_pos_index']),150, 150)

print(test_head_pos.shape)
print(train_head_pos.shape)

(1033, 150, 150)
(4222, 150, 150)


In [40]:
test_head_pos

array([[[ 5,  3,  3, ...,  0,  0,  0],
        [ 5,  3,  3, ...,  0,  0,  0],
        [ 5,  3,  3, ...,  0,  0,  0],
        ...,
        [ 5,  3,  3, ...,  0,  0,  0],
        [ 5,  3,  3, ...,  0,  0,  0],
        [ 5,  3,  3, ...,  0,  0,  0]],

       [[ 1,  1,  6, ...,  0,  0,  0],
        [ 1,  1,  6, ...,  0,  0,  0],
        [ 1,  1,  6, ...,  0,  0,  0],
        ...,
        [ 1,  1,  6, ...,  0,  0,  0],
        [ 1,  1,  6, ...,  0,  0,  0],
        [ 1,  1,  6, ...,  0,  0,  0]],

       [[ 9,  9,  5, ...,  0,  0,  0],
        [ 9,  9,  5, ...,  0,  0,  0],
        [ 9,  9,  5, ...,  0,  0,  0],
        ...,
        [ 9,  9,  5, ...,  0,  0,  0],
        [ 9,  9,  5, ...,  0,  0,  0],
        [ 9,  9,  5, ...,  0,  0,  0]],

       ...,

       [[ 2, 30,  7, ...,  0,  0,  0],
        [ 2, 30,  7, ...,  0,  0,  0],
        [ 2, 30,  7, ...,  0,  0,  0],
        ...,
        [ 2, 30,  7, ...,  0,  0,  0],
        [ 2, 30,  7, ...,  0,  0,  0],
        [ 2, 30,  7, ...,  0,  0

In [82]:
# split dataset

y = data['label']
X = train
x_train_metadata = train_head_pos

#train_ss is (4022,1)
x_train, x_val, x_metadata, x_metadata_val, x_ss, x_ss_val, y_train, y_val = train_test_split(
                            X, x_train_metadata, train_ss, y, random_state = 42, test_size = 0.2, shuffle = True)

print(x_ss.shape)
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(3377, 1)
(3377, 150, 768)
(3377,)
(845, 150, 768)
(845,)


In [53]:
import sys
sys.path.append('/home/alissa77/fake-news-detect/WWW2021/word-embedding')

def load_embeddings(language, embeddings_file):
    assert language in ['Chinese', 'English']

    embeddings_index = {}
    with open(embeddings_file, 'r') as f:
        lines = f.readlines()
        lines = [l.strip() for l in lines]

        lines = lines if language == 'English' else lines[1:]

        for line in lines:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, 'f', sep=' ')
            embeddings_index[word] = coefs

    print('File: {}, there are {} vectors'.format(
        embeddings_file, len(embeddings_index)))
    return embeddings_index

# embedding
MAX_NUM_WORDS = 6000
embeddings_index = load_embeddings(language='Chinese', embeddings_file='/home/alissa77/fake-news-detect/WWW2021/word-embedding/sgns.weibo.bigram-char')
CONTENT_WORDS = 100
EMBEDDING_DIM = 300

File: /home/alissa77/fake-news-detect/WWW2021/word-embedding/sgns.weibo.bigram-char, there are 195197 vectors


In [ ]:
texts=

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found {} unique tokens.'.format(len(word_index)))

content_arr = pad_sequences(sequences, maxlen=CONTENT_WORDS, padding='post')
print('Content Array: {}'.format(content_arr.shape))

num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.random.randn(num_words, EMBEDDING_DIM)

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Embedding Matrix: {}'.format(embedding_matrix.shape))

np.save('embedding_matrix.npy')

for i, t in enumerate(['train', 'val', 'test']):
    np.save(os.path.join(output_dir, '{}_{}.npy'.format(t, arrs[i].shape)), arrs[i])

# model

In [50]:
from keras.layers import Input
from keras.layers import Bidirectional
from keras.layers import GRU, TimeDistributed
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dense
from keras.layers import Embedding, Concatenate
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.initializers import Constant
from keras import backend as K
from keras.layers.core import Lambda


class EmotionEnhancedBiGRU:
    def __init__(self, max_sequence_length, embedding_matrix, emotion_dim=0, category_num=2, hidden_units=32, l2_param=0.01, lr_param=0.001):
        self.max_sequence_length = max_sequence_length
        self.embedding_matrix = embedding_matrix
        self.emotion_dim = emotion_dim
        self.hidden_units = hidden_units
        self.category_num = category_num
        self.l2_param = l2_param

        self.model = self.build()
        self.model.compile(loss='binary_crossentropy', optimizer=Adam(
            lr=lr_param, beta_1=0.8), metrics=['accuracy'])

    def build(self):
        semantic_input = Input(
            shape=(self.max_sequence_length,), name='word-embedding-input')

        semantic_emb = Embedding(self.embedding_matrix.shape[0],
                                 self.embedding_matrix.shape[1],
                                 embeddings_initializer=Constant(
                                     self.embedding_matrix),
                                 input_length=self.max_sequence_length,
                                 trainable=False)(semantic_input)

        gru = Bidirectional(
            GRU(self.hidden_units, return_sequences=True))(semantic_emb)
        avg_pool = GlobalAveragePooling1D()(gru)

        if self.emotion_dim != 0:
            emotion_input = Input(
                shape=(self.emotion_dim,), name='emotion-input')
            emotion_enhanced = Concatenate()([avg_pool, emotion_input])

            dense = Dense(32, activation='relu', kernel_regularizer=l2(
                self.l2_param))(emotion_enhanced)
            output = Dense(self.category_num, activation='softmax',
                           kernel_regularizer=l2(self.l2_param))(dense)
            model = Model(
                inputs=[semantic_input, emotion_input], outputs=output)
        else:
            dense = Dense(32, activation='relu',
                          kernel_regularizer=l2(self.l2_param))(avg_pool)
            output = Dense(self.category_num, activation='softmax',
                           kernel_regularizer=l2(self.l2_param))(dense)
            model = Model(inputs=[semantic_input], outputs=output)

        return model

In [83]:
sent_input = Input((150,768,), dtype='float32')
pos_input = Input((150,150,), dtype='float32')
concat = concatenate([sent_input, pos_input], axis = -1)
gru = Bidirectional(GRU(32, return_sequences=True))(concat)
avg_pool = GlobalAveragePooling1D()(gru)

ss_input = Input((1,),dtype ="float32")
emotion_enhanced = concatenate([avg_pool, ss_input], axis = -1)

dense = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(emotion_enhanced)
output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))(dense)
model = Model(inputs=[pos_input, sent_input, ss_input], outputs=output)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
filepath="BIGRU.hdf5"
mcp_save = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1, epsilon=1e-4, mode='min')

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_40 (InputLayer)           (None, 150, 768)     0                                            
__________________________________________________________________________________________________
input_41 (InputLayer)           (None, 150, 150)     0                                            
__________________________________________________________________________________________________
concatenate_22 (Concatenate)    (None, 150, 918)     0           input_40[0][0]                   
                                                                 input_41[0][0]                   
__________________________________________________________________________________________________
bidirectional_14 (Bidirectional (None, 150, 64)      182592      concatenate_22[0][0]       

In [84]:
print(x_train.shape)
print(x_metadata_train.shape)
print(x_ss.shape)

(3377, 150, 768)
(3377, 150, 150)
(3377, 1)


In [86]:
# test ans
hist = model.fit([x_train, x_metadata_train, x_ss], y_train, epochs=50, batch_size=32,
                validation_data=([x_val,  x_metadata_val, x_ss_val], y_val),
                callbacks=[earlyStopping, mcp_save, reduce_lr_loss]
                 )
model.save("BIGRU.h5")

ValueError: Error when checking input: expected input_41 to have shape (150, 150) but got array with shape (150, 768)

In [ ]:
# without multiple input acc: 0.85
# with pos acc: 0.87
model_path = "BIGRU.h5"
a = load_model(model_path, custom_objects={'AttentionLayer': AttentionLayer})
x_test = test
x_metadata = test_head_pos
y_test = df.label

loss, acc = a.evaluate([x_test, x_metadata, test_ss_pad] ,y_test, batch_size=64)
print("Test set accuracy: ",acc)
print("Test set loss: ", loss)

In [ ]:
# Plot of accuracy in each epoch
accu_curve=plt.figure()
plt.plot(hist.history['accuracy'],"r")
plt.plot(hist.history['val_accuracy'],"b")
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
accu_curve.savefig('accuracy_features.png')
plt.show()

# Plot of loss in each epoch
loss_curve = plt.figure()
plt.plot(hist.history['loss'],"r")
plt.plot(hist.history['val_loss'],"b")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
loss_curve.savefig('loss_features.png')
plt.show()

In [ ]:
import json
import jieba
import time
import tqdm

# save the result in /preprocess/data  folder
save_dir = './weibo20'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

def cut_words_from_text(text):
    return list(jieba.cut(text))
    
    
datasets_ch = ["Weibo-20"]

for dataset in datasets_ch :
    print('\n\n{} [{}]\tProcessing the dataset: {} {}\n'.format(
        '-'*20, time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()), dataset, '-'*20))

    data_dir = os.path.join('/home/alissa77/WWW2021/dataset', dataset)
    output_dir = os.path.join(save_dir, dataset)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    emotion_dir = os.path.join(output_dir, 'emotions')
    if not os.path.exists(emotion_dir):
        os.mkdir(emotion_dir)

    split_datasets = [json.load(open(os.path.join(data_dir, '{}.json'.format(t)), 'r')) 
                      for t in ['train', 'val', 'test']]
    split_datasets = dict(zip(['train', 'val', 'test'], split_datasets))

    for t, pieces in split_datasets.items():
        
        # words cutting
        for p in pieces:
            p['content_words'] = cut_words_from_text(p['content'])
            
        with open(os.path.join(output_dir, '{}.json'.format(t)), 'w') as f:
            json.dump(pieces, f, indent=4, ensure_ascii=False)
        '''
        emotion_arr = [extract_pkg.extract_dual_emotion(p) for p in tqdm(pieces)]
        emotion_arr = np.array(emotion_arr)
        print(emotion_arr)
        print('{} dataset: got a {} emotion arr'.format(t, emotion_arr.shape))
        np.save(os.path.join(emotion_dir, '{}_{}.npy'.format( t, emotion_arr.shape)), emotion_arr)
        '''    

In [45]:
data_dir = os.path.join('/home/alissa77/fake-news-detect/weibo20', dataset)
split_datasets = [json.load(open(os.path.join(data_dir, '{}.json'.format(t)), 'r')) for t in ['train', 'val', 'test']]
split_datasets = dict(zip(['train', 'val', 'test'], split_datasets))

for t, pieces in split_datasets.items():
    # words cutting
    for p in pieces:
        p["content_words"]

In [25]:
import os
import json
from tqdm import tqdm
import time
import jieba
import pandas as pd
import numpy as np
import sys
import joblib
import csv

datasets_ch = ['Weibo-16','Weibo-20',]

def cut_words_from_text(text):
    return list(jieba.cut(text))
def extract_publisher_emotion(content, content_words, emotions_dict):
    text, cut_words = content, content_words
    arr = np.zeros(55)

    arr[8:37] = dalianligong_arr(cut_words)
    arr[37:38] = boson_value(cut_words)
    arr[38:55] = auxilary_features(text, cut_words)

    return arr

def extract_dual_emotion(piece):
    publisher_emotion = extract_publisher_emotion(
        piece['content'], piece['content_words'], piece['content_emotions'])

    return publisher_emotion
def get_not_and_how_value(cut_words, i, windows):
    not_cnt = 0
    how_v = 1

    left = 0 if (i - windows) < 0 else (i - windows)
    window_text = ' '.join(cut_words[left:i])

    for w in negation_words:
        if w in window_text:
            not_cnt += 1
    for w in how_words_dict.keys():
        if w in window_text:
            how_v *= how_words_dict[w]

    return (-1) ** not_cnt, how_v


def boson_value(cut_words, windows=2):
    value = 0
    def b_edge(value):
        if value<0:
            return -4
        else:
            return 4
    valence = {}
    arousal = {}
    cvaw_dict = dict()    
    
    with open('/home/alissa77/WWW2021 copy/code/emotion/cvaw4.csv', newline='') as csvfile:
        rows = csv.DictReader(csvfile)
        for row in rows:
            valence[row['Word']] = (row['Valence_Mean'])
            arousal[row['Word']] = (row['Arousal_Mean'])
        #cvaw_dict[arousal[0]] = float(arousal[1])
        
    valence = {x: round(float(valence[x])-5, 2) for x in valence}
    arousal = {x: round(float(arousal[x])-5, 2) for x in arousal}
    #print('[CVAW]\t There are {} words'.format(len(cvaw_dict)))

    for i, word in enumerate(cut_words):
        if word in arousal:
            not_v, how_v = get_not_and_how_value(cut_words, i, windows)
            
            value += not_v * how_v * arousal[word]
            
    return value


# ============================== Category ==============================
baidu_emotions = ['angry', 'disgusting', 'fearful',
                  'happy', 'sad', 'neutral', 'pessimistic', 'optimistic']
baidu_emotions.sort()

baidu_emotions_2_index = dict(
    zip(baidu_emotions, [i for i in range(len(baidu_emotions))]))


def baidu_arr(emotions_dict):
    arr = np.zeros(len(baidu_emotions))

    if emotions_dict is None:
        return arr

    for k, v in emotions_dict.items():
        # like -> happy
        if k == 'like':
            arr[baidu_emotions_2_index['happy']] += v
        else:
            arr[baidu_emotions_2_index[k]] += v

    return arr

# ============================== Lexicon and Intensity ==============================
# load negation words
negation_words = []
with open('/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/others/negative/negationWords.txt', 'r') as src:
    lines = src.readlines()
    for line in lines:
        negation_words.append(line.strip())

print('\nThe num of negation words: ', len(negation_words))

# load degree words
how_words_dict = dict()
with open('/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/HowNet/intensifierWords.txt', 'r') as src:
    lines = src.readlines()
    for line in lines:
        how_word = line.strip().split()
        how_words_dict[' '.join(how_word[:-1])] = float(how_word[-1])

print('The num of degree words: ', len(how_words_dict),
      '. eg: ', list(how_words_dict.items())[0])


_, words2array = joblib.load(
    '/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/大连理工大学情感词汇本体库/preprocess/words2array_27351.pkl')
print('[Dalianligong]\tThere are {} words, the dimension is {}'.format(
    len(words2array), words2array['快乐'].shape))


def dalianligong_arr(cut_words, windows=2):
    arr = np.zeros(29)
    
    for i, word in enumerate(cut_words):
        if word in words2array:
            not_v, how_v = get_not_and_how_value(cut_words, i, windows)
            arr += not_v * how_v * words2array[word]

    return arr

# ============================== Auxilary Features ==============================
# Emoticon
emoticon_df = pd.read_csv(
    '/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/others/emoticon/emoticon.csv')
emoticons = emoticon_df['emoticon'].tolist()
emoticon_types = list(set(emoticon_df['label'].tolist()))
emoticon_types.sort()
emoticon2label = dict(
    zip(emoticon_df['emoticon'].tolist(), emoticon_df['label'].tolist()))
emoticon2index = dict(
    zip(emoticon_types, [i for i in range(len(emoticon_types))]))

print('[Emoticon]\tThere are {} emoticons, including {} categories'.format(
    len(emoticons), len(emoticon_types)))


def emoticon_arr(text, cut_words):
    arr = np.zeros(len(emoticon_types))

    if len(cut_words) == 0:
        return arr

    for i, emoticon in enumerate(emoticons):
        if emoticon in text:
            arr[emoticon2index[emoticon2label[emoticon]]
                ] += text.count(emoticon)

    return arr / len(cut_words)


# Punctuation
def symbols_count(text):
    excl = (text.count('！') + text.count('!')) / len(text)
    ques = (text.count('？') + text.count('?')) / len(text)
    comma = (text.count(',') + text.count('，')) / len(text)
    dot = (text.count('.') + text.count('。')) / len(text)
    ellip = (text.count('..') + text.count('。。')) / len(text)

    return excl, ques, comma, dot, ellip


# Sentimental Words??????? Hownet
def init_words(file):
    with open(file, 'r', encoding='utf-8') as src:
        words = src.readlines()
        words = [l.strip() for l in words]
    return list(set(words))

pos_words = init_words('/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/HowNet/正面情感词语（中文）.txt')
pos_words += init_words('/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/HowNet/正面评价词语（中文）.txt')
neg_words = init_words('/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/HowNet/负面情感词语（中文）.txt')
neg_words += init_words('/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/HowNet/负面评价词语（中文）.txt')
pos_words = set(pos_words)
neg_words = set(neg_words)
print('[HowNet]\tThere are {} positive words and {} negative words'.format(len(pos_words), len(neg_words)))

# count sentiment_words
def sentiment_words_count(cut_words):
    if len(cut_words) == 0:
        return [0, 0, 0, 0]

    # positive and negative words
    sentiment = []
    for words in [pos_words, neg_words]:
        c = 0
        for word in words:
            if word in cut_words:
                c += 1
        sentiment.append(c)
    sentiment = [c / len(cut_words) for c in sentiment]

    # degree words
    degree = 0
    for word in how_words_dict:
        if word in cut_words:
            degree += how_words_dict[word]

    # negation words
    negation = 0
    for word in negation_words:
        negation += cut_words.count(word)
    negation /= len(cut_words)
    sentiment += [degree, negation]

    return sentiment


# Personal Pronoun
first_pronoun = init_words(
    '/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/others/pronoun/1-personal-pronoun.txt')
second_pronoun = init_words(
    '/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/others/pronoun/2-personal-pronoun.txt')
third_pronoun = init_words(
    '/home/alissa77/fake-news-detect/WWW2021/resources/Chinese/others/pronoun/3-personal-pronoun.txt')
pronoun_words = [first_pronoun, second_pronoun, third_pronoun]


def pronoun_count(cut_words):
    if len(cut_words) == 0:
        return [0, 0, 0]

    pronoun = []
    for words in pronoun_words:
        c = 0
        for word in words:
            c += cut_words.count(word)
        pronoun.append(c)

    return [c / len(cut_words) for c in pronoun]


# Auxilary Features
def auxilary_features(text, cut_words):
    arr = np.zeros(17)
    arr[:5] = emoticon_arr(text, cut_words)
    arr[5:10] = symbols_count(text)
    arr[10:14] = sentiment_words_count(cut_words)
    arr[14:17] = pronoun_count(cut_words)

    return arr

for dataset in datasets_ch :
    print('\n\n{} [{}]\tProcessing the dataset: {} {}\n'.format(
        '-'*20, time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()), dataset, '-'*20))

    data_dir = os.path.join('/home/alissa77/fake-news-detect/preprocess_data', dataset)
    output_dir = os.path.join(save_dir, dataset)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    emotion_dir = os.path.join(output_dir, 'emotions')
    if not os.path.exists(emotion_dir):
        os.mkdir(emotion_dir)

    split_datasets = [pd.read_pickle(os.path.join(data_dir, '{}.pkl'.format(t))) for t in ['train', 'val', 'test']] 
    split_datasets = dict(zip(['train', 'val', 'test'], split_datasets))
    
    for t, p in split_datasets.items():
        emotion_arr = extract_dual_emotion(p) 


The num of negation words:  44
The num of degree words:  214 . eg:  ('百分之百', 2.0)
[Dalianligong]	There are 27351 words, the dimension is (29,)
[Emoticon]	There are 104 emoticons, including 5 categories
[HowNet]	There are 1899 positive words and 4315 negative words


-------------------- [2021-08-13 17:28:55]	Processing the dataset: Weibo-16 --------------------



KeyError: 'Requested level (！) does not match index name (None)'

In [26]:
!pwd

/home/alissa77/WWW2021 copy/code/preprocess/data
